In [1]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import emoji 

In [3]:
emoji_dict = {  "0" : "\u2764\uFE0F" ,
                "1" : ":baseball:" ,
                "2" : ":grinning_face_with_big_eyes:" ,
                "3" : ":disappointed_face:" ,
                "4" : ":fork_and_knife:" 
             }

In [4]:
import pandas as pd 
import numpy as np 

In [5]:
train = pd.read_csv('train_emoji.csv' , header = None )

In [6]:
test = pd.read_csv('test_emoji.csv' , header = None ) 

In [7]:
data = train.values 
xtrain = train[0]
ytrain = train[1]
xtest = test[0]
ytest = test[1] 

In [8]:
for i in range ( 5 ):
    print( xtrain[i] , emoji.emojize( emoji_dict[str(ytrain[i])]) )

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [9]:
f = open('glove.6B.50d.txt' , encoding='utf-8') 

In [10]:
embed_ind = {}
for line in f :
    values = line.split()
    word = values[0]
    coefs = np.asarray( values[1:] , dtype = 'float' )
    embed_ind [word] = coefs 
f.close()

In [11]:
emb_dim = embed_ind["eat"].shape[0]

In [12]:
def embed_out( x ):
    maxlen = 10  
    out = np.zeros( (x.shape[0], maxlen , emb_dim ) )
    for ix in range ( x.shape[0] ):
        x[ix] = x[ix].split()   
        for ij in range ( len(x[ix]) ):
            try:
                out[ix][ij] = embed_ind[ x[ix][ij].lower() ]
            except:
                out[ix][ij] = np.zeros( (50 ,) )
                
    return out
embed_matrix_train = embed_out( xtrain )
embed_matrix_test = embed_out( xtest )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
print( xtrain[0] )
print( len(xtrain[0]) )

['never', 'talk', 'to', 'me', 'again']
5


In [14]:
!pip install keras.utils   
from keras.utils.np_utils import to_categorical  
ytrain = to_categorical ( ytrain , num_classes = 5 )
ytest = to_categorical( ytest , num_classes = 5 )

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from keras.models import Sequential
from keras.layers import *

In [16]:
model = Sequential()
model.add( LSTM (64 , input_shape = (10,50),return_sequences = True ) )
model.add( Dropout(0.5) )
model.add( LSTM(64, return_sequences=False) )
model.add( Dropout(0.5) )
model.add( Dense(5) )
model.add( Activation ('softmax') )
model.compile( loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics= ['accuracy'] )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            29440     
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
 activation (Activation)     (None, 5)                 0         
                                                                 
Total params: 62,789
Trainable params: 62,789
Non-traina

In [21]:
hist = model.fit( embed_matrix_train , ytrain , epochs = 1000 , batch_size = 64 , shuffle = True , validation_split = 0.2  )

Epoch 1/1000
2/2 [==============================] - 0s 304ms/step - loss: 1.0437e-04 - accuracy: 1.0000 - val_loss: 4.3993 - val_accuracy: 0.5556
Epoch 2/1000
2/2 [==============================] - 0s 149ms/step - loss: 1.5261e-04 - accuracy: 1.0000 - val_loss: 4.3983 - val_accuracy: 0.5556
Epoch 3/1000
2/2 [==============================] - 0s 193ms/step - loss: 8.2228e-05 - accuracy: 1.0000 - val_loss: 4.3976 - val_accuracy: 0.5556
Epoch 4/1000
2/2 [==============================] - 0s 149ms/step - loss: 9.8422e-05 - accuracy: 1.0000 - val_loss: 4.3974 - val_accuracy: 0.5556
Epoch 5/1000
2/2 [==============================] - 0s 161ms/step - loss: 6.7942e-05 - accuracy: 1.0000 - val_loss: 4.3970 - val_accuracy: 0.5556
Epoch 6/1000
2/2 [==============================] - 0s 186ms/step - loss: 7.3975e-05 - accuracy: 1.0000 - val_loss: 4.3966 - val_accuracy: 0.5556
Epoch 7/1000
2/2 [==============================] - 0s 155ms/step - loss: 2.7293e-04 - accuracy: 1.0000 - val_loss: 4.3970 -

In [22]:
model.evaluate( embed_matrix_test , ytest )

2/2 [==============================] - 0s 9ms/step - loss: 4.4688 - accuracy: 0.5893


[4.468841552734375, 0.5892857313156128]

In [23]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h5" , monitor = 'accuracy' , verbose = True , save_best_only = True )
earlystop = EarlyStopping( monitor = 'val_acc' , patience = 10 )  
hist = model.fit( embed_matrix_train , ytrain , epochs = 1000 , batch_size = 64 , shuffle = True , validation_split = 0.2  )

Epoch 1/1000
2/2 [==============================] - 0s 87ms/step - loss: 4.2733e-05 - accuracy: 1.0000 - val_loss: 4.2872 - val_accuracy: 0.5926
Epoch 2/1000
2/2 [==============================] - 0s 42ms/step - loss: 1.1006e-04 - accuracy: 1.0000 - val_loss: 4.2898 - val_accuracy: 0.5926
Epoch 3/1000
2/2 [==============================] - 0s 46ms/step - loss: 3.7327e-05 - accuracy: 1.0000 - val_loss: 4.2917 - val_accuracy: 0.5926
Epoch 4/1000
2/2 [==============================] - 0s 47ms/step - loss: 4.0318e-05 - accuracy: 1.0000 - val_loss: 4.2942 - val_accuracy: 0.5926
Epoch 5/1000
2/2 [==============================] - 0s 49ms/step - loss: 3.7042e-05 - accuracy: 1.0000 - val_loss: 4.2962 - val_accuracy: 0.5926
Epoch 6/1000
2/2 [==============================] - 0s 43ms/step - loss: 4.2307e-05 - accuracy: 1.0000 - val_loss: 4.2981 - val_accuracy: 0.5926
Epoch 7/1000
2/2 [==============================] - 0s 46ms/step - loss: 1.2542e-04 - accuracy: 1.0000 - val_loss: 4.3009 - val_ac

In [24]:

model.evaluate( embed_matrix_test , ytest )

2/2 [==============================] - 0s 9ms/step - loss: 4.4484 - accuracy: 0.6250


[4.448363780975342, 0.625]